In [11]:
import sys  
sys.path.insert(0, './../')
import numpy as np
import bbdc2021 as bbdc
import pandas as pd
from importlib import reload
reload(bbdc)
import tensorflow as tf
from keras import layers
import keras
from keras import backend as K
import matplotlib.pyplot as plt
from matplotlib import colors 
import evaluation.evaluate as evaluate

In [3]:
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
X_train, Y_train, trainFileList = bbdc.load_audioset("long_unbalanced_train_segments.csv", "unbalanced_fft_mel_l04_o02495_f32/", pathToDataDir="./../google_audioset/googleData/fft/")
X_validation, Y_validation, validationFileList = bbdc.load_audioset("long_balanced_train_segments.csv", "balanced_fft_mel_l04_o02495_f32/", pathToDataDir="./../google_audioset/googleData/fft/")
X_test, Y_test, testFileList = bbdc.load_audioset("long_eval_segments.csv", "eval_fft_mel_l04_o02495_f32/", pathToDataDir="./../google_audioset/googleData/fft/")

1235it [00:40, 29.93it/s]

In [9]:
type(X_test)

list

In [7]:
channels = [40, 50]
numClasses=13

input_layer = layers.Input(shape=(None, 32))
x = input_layer
x = layers.Conv1D(channels[0], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.Conv1D(channels[0], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.Conv1D(channels[1], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.Conv1D(channels[1], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.Conv1D(channels[0], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv1D(numClasses, kernel_size=1, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Activation(keras.activations.sigmoid)(x)

model = keras.models.Model(inputs=input_layer, outputs=x)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 32)]        0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 40)          3880      
_________________________________________________________________
conv1d_9 (Conv1D)            (None, None, 40)          4840      
_________________________________________________________________
batch_normalization_3 (Batch (None, None, 40)          160       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 40)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, None, 50)          6050      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, None, 50)          7550

In [10]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['mae', 'accuracy'])
history = model.fit(np.array(X_test), np.array(Y_test), batch_size=1, epochs=10, shuffle=True)

Epoch 1/10
585/585 [==============================] - 12s 5ms/step - loss: 9.4214 - mae: 0.8321 - accuracy: 0.8584
Epoch 2/10
585/585 [==============================] - 3s 5ms/step - loss: 39.5332 - mae: 0.8453 - accuracy: 1.0000
Epoch 3/10
585/585 [==============================] - 3s 5ms/step - loss: 77.1881 - mae: 0.8461 - accuracy: 1.0000
Epoch 4/10
585/585 [==============================] - 3s 5ms/step - loss: 87.3681 - mae: 0.8461 - accuracy: 1.0000
Epoch 5/10
585/585 [==============================] - 3s 5ms/step - loss: 109.5343 - mae: 0.8461 - accuracy: 1.0000
Epoch 6/10
585/585 [==============================] - 3s 5ms/step - loss: 143.7923 - mae: 0.8459 - accuracy: 1.0000
Epoch 7/10
585/585 [==============================] - 3s 5ms/step - loss: 179.6251 - mae: 0.8461 - accuracy: 1.0000
Epoch 8/10
585/585 [==============================] - 3s 4ms/step - loss: 192.0517 - mae: 0.8459 - accuracy: 1.0000
Epoch 9/10
585/585 [==============================] - 3s 5ms/step - loss: 24

In [14]:
X_test.dtype

dtype('O')